In [ ]:
%%capture
!pip install --upgrade kaggle_environments

In [ ]:
from kaggle_environments import make
env = make("kore_fleets", debug=True)
env.run(["../input/kore-intro-iv-combat-unofficial/attacker.py", "../input/kore-intro-iv-combat-unofficial/attacker2.py"])
env.render(mode="ipython", width=700, height=700)

pixyz  
last update 2022 05 20  
ゆっくりしていってね！  

# Contents

* v0 [**Official guide**](#Officlal_guide)

* v1 [**Use fleets informetion (Interception agent)**](#Use-fleets-informetion-(Interception-agent))

* v2 [**Submission form**](#Submission-form)

こっちも観てね！

Kore Intro I: The Basics unofficial 実況  
https://www.kaggle.com/code/pixyz0130/kore-intro-i-the-basics

Kore Intro II: Mining Kore unofficial 実況  
https://www.kaggle.com/code/pixyz0130/kore-intro-ii-mining-kore

Kore Intro III: Expanding the Empire! unofficial　実況  
https://www.kaggle.com/code/pixyz0130/kore-intro-iii-expanding-the-empire

**霊夢:今回はコンペ理解のために、主催者のBovard氏が書いてくれたコードを日本語訳してみたよ。今回はIntro Ⅳをやっていくよ。**

**魔理沙:他に気になったことがあったら突っ込んでいくぞ。**

**Reimu: This time, I translated the code written by the organizer into Japanese to understand the competition.**

**Marisa: If you have any other concerns, I'll dig in.**

<img src="https://4.bp.blogspot.com/-uoVuBWIbdiA/WvQHqpx_YCI/AAAAAAABL8g/NiFZ6K71VBc_0_dcKb3_4nhnvFJ_JMNuACLcBGAs/s450/network_dennou_sekai_figure.png" width = 300>

# Official guide

**霊夢:まずは公式ガイドの日本語訳からやってくよ**  

**Reimu: Let's start with the Japanese translation of the official guide**
https://www.kaggle.com/code/bovard/kore-intro-iv-combat

## Welcome back Commander!

We will take a look at combat here!

ここで戦闘を見ていきます！

## Fleet to fleet combat

When two fleets move to occupy the same tile or move orthogonally adjacent to an enemy fleet they do damage to eachother!

2つの艦隊が同じタイルを占めるように移動したり、敵の艦隊に隣接して直交して移動したりすると、お互いにダメージを与えます。

#### Same tile
The largest fleet (in ship count) wins, surviving with `intial_ships - second_largest_fleet_ships` ships left. For example, if fleets from 3 different players (1, 2, 3) enter the same tile with f1 having 100 ships, f2 having 50, and f3 having 25, f1 would survive with 100 - 50 = 50 ships. The surviving fleet, if there is one, takes the kore from all the other fleets.

（船数で）最大の艦隊が勝利し、`(勝利した艦隊の船の数)-（2番目に多かった船の数）`の船が残ったまま生き残ります。 たとえば、3人の異なるプレイヤー（1、2、3）の艦隊が同じタイルに入り、f1が100隻、f2が50隻、f3が25隻の場合、f1は100-50=50隻で生き残ります。 生き残った艦隊は、もしあれば、他のすべての艦隊から資源を奪います。

#### Adjacent tiles
After a fleet moves and resolves and on-tile collisions, it does damage to all adjacent tiles equal to its ship count. For example, if two fleets of different players of sizes 100 and 50 end up adjacent, the 100 ship fleet will survive with only 50 ships left. Note that a single fleet can do damage to up to 3 other fleets if positioned correctly!

艦隊が移動して解決し、タイル上で衝突した後、隣接するすべてのタイルにその船の数に等しいダメージを与えます。 たとえば、サイズ100と50の異なるプレーヤーの2つの艦隊が隣接することになった場合、100隻の艦隊は、残りの50隻だけで生き残ります。 正しく配置されている場合、1つの艦隊が最大3つの他の艦隊にダメージを与える可能性があることに注意してください！

Let's see it in action! With the [starter bot](https://www.kaggle.com/code/bovard/basic-python-starter-notebook).

In [ ]:
%%writefile starter.py
# from https://github.com/Kaggle/kaggle-environments/blob/master/kaggle_environments/envs/kore_fleets/starter_bots/python/main.py
   
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint

def agent(obs, config):
    board = Board(obs, config)
    me=board.current_player

    me = board.current_player
    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    kore_left = me.kore

    for shipyard in me.shipyards:
        if shipyard.ship_count > 10:
            direction = Direction.from_index(turn % 4)
            action = ShipyardAction.launch_fleet_with_flight_plan(randint(2,3), direction.to_char())
            shipyard.next_action = action
        elif kore_left > spawn_cost * shipyard.max_spawn:
            action = ShipyardAction.spawn_ships(shipyard.max_spawn)
            shipyard.next_action = action
            kore_left -= spawn_cost * shipyard.max_spawn
        elif kore_left > spawn_cost:
            action = ShipyardAction.spawn_ships(1)
            shipyard.next_action = action
            kore_left -= spawn_cost

    return me.next_actions

In [ ]:
starter_agent_path = "/kaggle/working/starter.py"
env.run([starter_agent_path, starter_agent_path, starter_agent_path, starter_agent_path])
env.render(mode="ipython", width=1000, height=800)

As you can see above, there is a lot of collisions and adjacent damage.

上記のように、多くの衝突と隣接する損傷があります。

**霊夢:同じマスを占領したときの戦闘と、隣接したときの戦闘が一つの艦隊に対して同時に起こった場合、どっちが先に処理されるんだ？**

**Reimu: If a battle when occupying the same square and a battle when adjacent to each other occur at the same time against one fleet, which one will be processed first?**

## Capturing a Shipyard

In addition to collisions, you can also capture an enemy shipyard. The same mechanics are used in collision - the larger fleet wins.

When you capture a shipyard the ship production rate of that shipyard is reset to 1!

Let's write a bot that amasses 50 ships and attempts to take over nearby shipyards. Note that we'll change our spawn code to take advantage of a shipyards increased production rate `shipyard.max_spawn`! This will allow us to spawn up to 10 ships a turn if we hold onto the shipyard long enough

衝突に加えて、敵の造船所を占領することもできます。 同じメカニズムが衝突で使用されます-より大きな艦隊が勝ちます。

造船所を占領すると、その造船所の船の生産率は1にリセットされます！

50隻の船を集め、近くの造船所を乗っ取ろうとするボットを書いてみましょう。 造船所の生産率が向上した`shipyard.max_spawn`を利用するために、スポーンコードを変更することに注意してください。 これにより、造船所を十分に長く保持している場合、1ターンに最大10隻の船をスポーンすることができます。

In [ ]:
%%writefile attacker.py
# from https://github.com/Kaggle/kaggle-environments/blob/master/kaggle_environments/envs/kore_fleets/starter_bots/python/main.py
   
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint

def get_closest_enemy_shipyard(board, position, me):
    min_dist = 1000000
    enemy_shipyard = None
    for shipyard in board.shipyards.values():
        if shipyard.player_id == me.id:
            continue
        dist = position.distance_to(shipyard.position, board.configuration.size)
        if dist < min_dist:
            min_dist = dist
            enemy_shipyard = shipyard
    return enemy_shipyard


def agent(obs, config):
    board = Board(obs, config)
    me=board.current_player

    me = board.current_player
    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    kore_left = me.kore

    for shipyard in me.shipyards:
        action = None
        if turn % 100 < 20 and shipyard.ship_count >= 50:
            closest_enemy_shipyard = get_closest_enemy_shipyard(board, shipyard.position, me)
            if not closest_enemy_shipyard:
                continue
            enemy_pos = closest_enemy_shipyard.position
            my_pos = shipyard.position
            flight_plan = "N" if enemy_pos.y > my_pos.y else "S"
            flight_plan += str(abs(enemy_pos.y - my_pos.y) - 1)
            flight_plan += "W" if enemy_pos.x < my_pos.x else "E"
            action = ShipyardAction.launch_fleet_with_flight_plan(shipyard.ship_count, flight_plan)            
        elif shipyard.ship_count >= 10 and turn % 7 == 0 and turn % 100 > 20 and turn % 100 < 90:
            direction = Direction.from_index(turn % 4)
            opposite = direction.opposite()
            flight_plan = direction.to_char() + "9" + opposite.to_char()
            action = ShipyardAction.launch_fleet_with_flight_plan(10, flight_plan)
        elif kore_left > spawn_cost * shipyard.max_spawn:
            action = ShipyardAction.spawn_ships(shipyard.max_spawn)
            kore_left -= spawn_cost * shipyard.max_spawn
        elif kore_left > spawn_cost:
            action = ShipyardAction.spawn_ships(1)
            kore_left -= spawn_cost
        shipyard.next_action = action

    return me.next_actions

In [ ]:
starter_agent_path = "/kaggle/working/starter.py"
attacker_agent_path = "/kaggle/working/attacker.py"
steps = env.run([attacker_agent_path, starter_agent_path, starter_agent_path, starter_agent_path])
env.render(mode="ipython", width=1000, height=800)

**霊夢:env.runを変数にstepsに代入して、中身を見てみるよ。**

**Reimu: Substitute env.run into a variable for steps and take a look at the contents.**

In [ ]:
print(steps[100][0])

In [ ]:
print(steps[100][1])
print(steps[100][2])
print(steps[100][3])

### Victory! 

Notice the number in the bottom right hand corner of the shipyard after you take it over (which displays `shipyard.max_spawn`) resetting to 1 and slowly incrementing when you take over a shipyard.

Good luck commander, and happy coding!

造船所を乗っ取った後、造船所の右下隅にある番号（ `shipyard.max_spawn`が表示されます）が1にリセットされ、造船所を乗っ取るとゆっくりと増加することに注意してください。

幸運の司令官、そして幸せなコーディング！

# Turn Resolution

**霊夢:戦闘とか合体とか色々な処理があることが分かったけど1ターンの中でどの順番で処理が実行されていくんだろう？**

**魔理沙:ルールのところにいろいろ書いてあるね。順番は下のようになってるぜ**

1. スポーンと起動
2. 艦隊は飛行計画を更新
3. 艦隊は造船所に変換
4. 艦隊の移動
5. 連合艦隊合体
6. 艦隊の衝突
7. 造船所の衝突
8. 艦隊から艦隊へのダメージ
9. 資源の発掘

**霊夢:なるほど、この順番がわからないと実際にコードが書けないから大切だね。**

**魔理沙:次回は、実際に主催者が出しているAgentの一例をみていくぜ。**

**Reimu: I found out that there are various processes such as battle and coalescence, but in what order will the processes be executed in one turn?**

**Marisa: There are a lot of things written in the rules. The order is as below**

1. Spawn and launch
2. Fleet updates flight plan
3. Fleet converted to shipyard
4. Fleet movement
5. Combined Fleet
6. Fleet clash
7. Shipyard collision
8. Damage from fleet to fleet
9. Excavation of resources

**Reimu: I see, it's important because you can't actually write code without knowing this order.**

**Marisa: Next time, let's take a look at an example of an Agent actually issued by the organizer.**

# Use fleets informetion (Interception agent)

**霊夢:さっきは、4人対戦をしていたので、2人で対戦させてみるよ。**

**Reimu: I was playing against four people earlier, so let's play against them.**

In [ ]:
steps = env.run([attacker_agent_path, starter_agent_path])
env.render(mode="ipython", width=1000, height=800)

In [ ]:
print(steps[226][0])

**霊夢:敵の艦隊の情報を取得することはできないのかな？**

**魔理沙:playersが艦隊と造船所の情報を表しているみたいだな。**

**霊夢:これを使って狙った敵の艦隊に攻撃するagentを作ってみよう**

**Reimu: Is it possible to get information about the enemy fleet?**

**Marisa: It looks like players are showing information about the fleet and shipyards.**

**Reimu: Let's use this to create an agent that attacks the targeted enemy fleet**

<img src="https://4.bp.blogspot.com/-dzQXmBcfN44/V_-wZ8tyQlI/AAAAAAAA-2Y/56siI2rWqt8O6afKaqVbNnEKXhQrkgWoACLcB/s180-c/war_character_senkan.png" width = 200>

**造船所に攻撃しようとする艦隊を迎撃するagentを作ります。**

**get_closest_enemy_fleetは、自分の造船所を攻撃しようとしている艦隊を検出し、その艦隊の場所と船の数を返す関数です**

**board.fleets.values()で艦隊の情報を取得します**

**それぞれのfleetは以下の情報を持っています**

**get_closest_enemy_fleet is a function that detects the fleet that is trying to attack your shipyard and returns the location and number of ships of that fleet**

**Get fleet information with board.fleets.values()**

**Each fleet has the following information**

player_id (どのプレイヤーの艦隊か)  
direction (向き)  
flight_plan (飛行計画)  
position (場所)  
ship_count (船の数)  

In [ ]:
%%writefile attacker2.py
# from https://github.com/Kaggle/kaggle-environments/blob/master/kaggle_environments/envs/kore_fleets/starter_bots/python/main.py
   
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint

def build_flight_plan(dir_idx, size):
    flight_plan = ""
    for i in range(4):
        flight_plan += Direction.from_index((dir_idx + i) % 4).to_char()
        if not i == 3:
            flight_plan += str(size)
    return flight_plan

def get_closest_enemy_fleet(board, position, me):
    enemy_fleet = [-1,-1,-1]
    for fleet in board.fleets.values():
        if fleet.player_id == me.id:
            continue
        plan = fleet.flight_plan
        dic = str(fleet.direction)[0]
        y = [fleet.position.y]
        x = [fleet.position.x]
        if len(plan) == 0:
            plan = dic
        for i in range(11):
            if plan[0] == 'N':
                y += [(y[i] - 1) % board.configuration.size]
                x += [x[i]]
            elif plan[0] == 'S':
                y += [(y[i] + 1) % board.configuration.size]
                x += [x[i]]
            elif plan[0] == 'W':
                x += [(x[i] - 1) % board.configuration.size]
                y += [y[i]]
            elif plan[0] == 'E':
                x += [(x[i] + 1) % board.configuration.size]
                y += [y[i]]
            elif dic == 'N':
                y += [(y[i] - 1) % board.configuration.size]
                x += [x[i]]
            elif dic == 'S':
                y += [(y[i] + 1) % board.configuration.size]
                x += [x[i]]
            elif dic == 'W':
                x += [(x[i] - 1) % board.configuration.size]
                y += [y[i]]
            elif dic == 'E':
                x += [(x[i] + 1) % board.configuration.size]
                y += [y[i]]

            if not len(plan) == 1:
                if plan[0].isdecimal() and not plan[0] == '1':                
                    plan = str(int(plan[0])-1) + plan[1:]
                else:
                    plan = plan[1:]
            if y[i+1] == position.y and x[i+1] == position.x:
                enemy_fleet = [y[(i+1)//2],x[(i+1)//2],fleet.ship_count]
                break
        
        if not enemy_fleet[0] == -1:
            break
    return enemy_fleet


def agent(obs, config):
    board = Board(obs, config)
    me=board.current_player

    me = board.current_player
    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    kore_left = me.kore
    for shipyard in me.shipyards:
        max_spawn = shipyard.max_spawn
        enemy_fleet = get_closest_enemy_fleet(board, shipyard.position, me)
        action = None
        if not enemy_fleet[0] == -1 and shipyard.ship_count + 1 > enemy_fleet[2]:
            enemy_pos_y = enemy_fleet[0]
            enemy_pos_x = enemy_fleet[1]            
            my_pos = shipyard.position
            flight_plan = ""
            if not enemy_pos_y == my_pos.y: flight_plan += "N" if enemy_pos_y > my_pos.y else "S"
            if not enemy_pos_y == my_pos.y: flight_plan += str(abs(enemy_pos_y - my_pos.y) - 1)
            if not enemy_pos_x == my_pos.x: flight_plan += "W" if enemy_pos_x < my_pos.x else "E"
            if not enemy_pos_x == my_pos.x: flight_plan += str(abs(enemy_pos_x - my_pos.x) - 1)
            if not enemy_pos_y == my_pos.y: flight_plan = "S" if enemy_pos_y > my_pos.y else "N"
            if not enemy_pos_y == my_pos.y: flight_plan += str(abs(enemy_pos_y - my_pos.y) - 1)
            if not enemy_pos_x == my_pos.x: flight_plan += "E" if enemy_pos_x < my_pos.x else "W"
            action = ShipyardAction.launch_fleet_with_flight_plan(enemy_fleet[2] + 2, flight_plan)
            
        elif shipyard.ship_count >= 10 and turn % 7 == 0 and turn % 100 > 20 and turn % 100 < 90:
            direction = Direction.from_index(turn % 4)
            opposite = direction.opposite()
            flight_plan = direction.to_char() + "9" + opposite.to_char()
            action = ShipyardAction.launch_fleet_with_flight_plan(10, flight_plan)
        elif kore_left > spawn_cost * shipyard.max_spawn:
            action = ShipyardAction.spawn_ships(shipyard.max_spawn)
            kore_left -= spawn_cost * shipyard.max_spawn
        elif kore_left > spawn_cost:
            action = ShipyardAction.spawn_ships(1)
            kore_left -= spawn_cost
        shipyard.next_action = action

    return me.next_actions

In [ ]:
attacker_agent2_path = "/kaggle/working/attacker2.py"

In [ ]:
steps = env.run([attacker_agent_path, attacker_agent2_path])
env.render(mode="ipython", width=1000, height=800)

**霊夢:ちゃんと迎撃してるね。でも迎撃することにメリットあるの？**

**魔理沙:相手の艦隊を破壊するとKoreを奪えること、少しでも造船所を占拠される時間を減らせることかな。試しに、迎撃をしないパターンもやってみよう**

**Reimu: You're intercepting properly. But is there any merit in intercepting?**

**Marisa: If you destroy the opponent's fleet, you can take Kore, and you can reduce the time to occupy the shipyard as much as possible. Let's try a pattern that does not intercept**

In [ ]:
%%writefile no_attacker.py
# from https://github.com/Kaggle/kaggle-environments/blob/master/kaggle_environments/envs/kore_fleets/starter_bots/python/main.py
   
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint

def build_flight_plan(dir_idx, size):
    flight_plan = ""
    for i in range(4):
        flight_plan += Direction.from_index((dir_idx + i) % 4).to_char()
        if not i == 3:
            flight_plan += str(size)
    return flight_plan


def agent(obs, config):
    board = Board(obs, config)
    me=board.current_player

    me = board.current_player
    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    kore_left = me.kore
    for shipyard in me.shipyards:
        max_spawn = shipyard.max_spawn
        action = None
                    
        if shipyard.ship_count >= 10 and turn % 7 == 0:
            direction = Direction.from_index(turn % 4)
            opposite = direction.opposite()
            flight_plan = direction.to_char() + "9" + opposite.to_char()
            action = ShipyardAction.launch_fleet_with_flight_plan(10, flight_plan)
        elif kore_left > spawn_cost * shipyard.max_spawn:
            action = ShipyardAction.spawn_ships(shipyard.max_spawn)
            kore_left -= spawn_cost * shipyard.max_spawn
        elif kore_left > spawn_cost:
            action = ShipyardAction.spawn_ships(1)
            kore_left -= spawn_cost
        shipyard.next_action = action

    return me.next_actions

In [ ]:
no_attacker_agent_path = "/kaggle/working/no_attacker.py"

In [ ]:
steps = env.run([attacker_agent_path, no_attacker_agent_path])
env.render(mode="ipython", width=1000, height=800)

**霊夢:迎撃してないと、やられちゃってるね。**

**魔理沙:そもそも、造船所に攻撃することのメリットがあまりないのかもしれないな。**

**Reimu: If you don't intercept, you'll be killed.**

**Marisa: In the first place, there may not be much merit in attacking the shipyard.**

# Submission form

**霊夢:submissionはpythonコードか、複数のコードをzipファイルでまとめて提出するかの2通りがあるんだね。**

**魔理沙:試しに、main.pyとsub.pyの2つのコードを使って、agentを作成してみよう**

**Reimu: There are two ways to submit: python code or submitting multiple codes together in a zip file.**

**Marisa: Let's try creating an agent using two codes, main.py and sub.py**

In [ ]:
%%writefile sub.py
# from https://github.com/Kaggle/kaggle-environments/blob/master/kaggle_environments/envs/kore_fleets/starter_bots/python/main.py
   
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint

def launch(agent):
    for shipyard in agent.shipyards:
        if shipyard.next_action:
            continue
        if shipyard.ship_count > 10:
            direction = Direction.from_index(randint(0,3))
            action = ShipyardAction.launch_fleet_with_flight_plan(randint(2,3), direction.to_char())
            shipyard.next_action = action
    return
    
def spawn(agent,spawn_cost,kore_left):
    for shipyard in agent.shipyards:
        if shipyard.next_action:
            continue
        if kore_left > spawn_cost * shipyard.max_spawn:
            action = ShipyardAction.spawn_ships(shipyard.max_spawn)
            shipyard.next_action = action
            kore_left -= spawn_cost * shipyard.max_spawn
        elif kore_left > spawn_cost:
            action = ShipyardAction.spawn_ships(1)
            shipyard.next_action = action
            kore_left -= spawn_cost

    return
    

In [ ]:
%%writefile main.py
# from https://github.com/Kaggle/kaggle-environments/blob/master/kaggle_environments/envs/kore_fleets/starter_bots/python/main.py
   
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint
from sub import launch,spawn


def agent(obs, config):
    board = Board(obs, config)
    
    me = board.current_player
    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    kore_left = me.kore
        
    launch(me)
    spawn(me,spawn_cost,kore_left)

    return me.next_actions

**sub.pyにlaunch,spawnという関数を定義して、main.pyの方でimportします。**

**Define launch and spawn functions in sub.py and import them in main.py.**

In [ ]:
steps = env.run(["./main.py"])
env.render(mode="ipython", width=500, height=400)

**霊夢:きちんと動いたね。**

**魔理沙:こうやってimportで関数を別のコードから呼び出すこともできるぜ。**

**Reimu: It worked.**

**Marisa: You can also call a function from another code with import like this.**

**霊夢:agentってグローバル関数とかを使って、一回計算した結果を次のターンに保存しておくことはできるのかな？**

**魔理沙:どうだろうか、実際に試してみよう。**

**Reimu: Is it possible to save the result of one calculation in the next turn by using a global function or something like agent?**

**Marisa: Let's try it out.**

In [ ]:
%%writefile starter2.py
# from https://github.com/Kaggle/kaggle-environments/blob/master/kaggle_environments/envs/kore_fleets/starter_bots/python/main.py
   
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint

history = 0

def agent(obs, config):
    global history
    board = Board(obs, config)
    me=board.current_player

    me = board.current_player
    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    kore_left = me.kore
    for shipyard in me.shipyards:
        if shipyard.ship_count > 10:
            direction = Direction.from_index(history % 4)
            action = ShipyardAction.launch_fleet_with_flight_plan(randint(2,3), direction.to_char())
            shipyard.next_action = action
            history += 1
        elif kore_left > spawn_cost * shipyard.max_spawn:
            action = ShipyardAction.spawn_ships(shipyard.max_spawn)
            shipyard.next_action = action
            kore_left -= spawn_cost * shipyard.max_spawn
        elif kore_left > spawn_cost:
            action = ShipyardAction.spawn_ships(1)
            shipyard.next_action = action
            kore_left -= spawn_cost

    return me.next_actions

In [ ]:
steps = env.run(["./starter2.py"])
env.render(mode="ipython", width=1000, height=800)

**霊夢:どうやらグローバル変数の中身は、turnをまたいでも継続されるみたいだね。**

**Reimu: Apparently, the contents of global variables continue across turns.**